In [823]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [824]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import pycountry

import pdfplumber

from collections import defaultdict


In [479]:
os.getcwd()
#os.chdir("/Users/veesheen/Desktop")

'/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024'

In [825]:
countries=list(pycountry.countries)

TypeError: 'list' object is not callable

In [826]:
countries[0].alpha_3

'ABW'

In [827]:
# missing KSA, VIE, INA, OMA, QAT, SRI, UAE, TPE, NEP, PLE, KUW, PHI, MAS

list_countries=[]

for i in range(len(countries)):
    
    list_countries.append(countries[i].alpha_3)

In [828]:
list_countries = list_countries + ['IRI', 'KSA', 'VIE', 'INA', 'OMA', 'QAT', 'SRI', 'UAE', 'TPE', 'NEP', 'PLE', 'KUW', 'PHI', 'MAS']

In [892]:
# Scrape U20 2024 PDFs

# TODO : correct data for first pole vault event, correct algo for picking up and correcting corrupted names

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/U20/All Results.pdf"

 
# Iterate over files in directory

event_pattern ='(MEN U20 -)|(WOMEN U20 -)'

heat_pattern = 'HEAT \d'

school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
year_pattern='\d\d\d\d'

#name_pattern='\d\d\d.{10-29}\d\d\d\d'

stage_pattern='FINAL|QUALIFYING|ROUND|SEMI|GROUP'

country_code = '\s[A-Z]{3}\s'

DOB_pattern='\s20[0-9]{2}\s'

#tag_pattern='\s[0-9]{3}\s'

tag_pattern='[0-9]{3}\s'

last_name_pattern = '[A-Z]{2,}'

wind_pattern='(\+\d\.\d|\-\d\.\d)'

name_pattern='[a-zA-Z\-\. \"]{3,31}'

results_begin = '^\d'  # line begins with a digit

react_time = '0\.[0-9]{3}'

rank_pattern = '^\d |^\dd '

qualification_pattern = ' Q | q '

relay_pattern = '\d\d\:\d\d\.\d\d'

corrupted_names = '[A-Z0-9]{5,}'

timing_pattern = '\d\d\.\d\d|\d.\d\d'




df_table=pd.DataFrame()  # initialize empty master df

df=None

final_row =[]

df_results=pd.DataFrame(columns = ['EVENT', 'COMPETITION', 'RANK', 'TAG_ID', 'NAME', 'DOB', 'COUNTRY', 'RESULT', 'REMARKS', 'GENDER', 'POINTS', 'QUALIFICATION', 'WIND', 'REACTION', 'HEAT', 'DICT_RESULT'])



with pdfplumber.open(file) as pdf:
        
        
    for i in range(len(pdf.pages)):    
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()        
        
        df=pd.DataFrame(text.splitlines())
        
                        
        for index, row in df.iterrows(): # find row containing event details and column names  
            
            # Look for corrupted names and correct
            
            #corrupted = re.findall(corrupted_names, row[0])
            
            #print('corrupted', index, corrupted)
            
            row[0]=row[0].replace('M2a0d0u5shani HERSARTIH', 'Madushani HERSARTIH 2005 SRI')
            row[0]=row[0].replace('Kama2l 0A0Z8IRA', 'Kamal AZIRA 2008')
            row[0]=row[0].replace('AL SH2E0K0I5LI', 'AL SHEKILI 2005')
            row[0]=row[0].replace('ALDHUH20O0O7RI', 'ALDHUHOORI 2007')
            row[0]=row[0].replace('KUWA20R0I7', 'KUWARI 2007')
            row[0]=row[0].replace('KARLSSON2007', 'KARLSSON 2007')
            row[0]=row[0].replace('EIDIYANKAKHAK2I005', 'EIDIYANKAKHAKI 2005')
            row[0]=row[0].replace('AGHAEI2005', 'AGHEI 2005')
            row[0]=row[0].replace('SWAKAR2007', 'SWAKAR 2007')
            row[0]=row[0].replace('AKALANKA2006', 'AKALANKA 2006')
            row[0]=row[0].replace('ZADJA2LI005', 'ZADJALI 2005')
            row[0]=row[0].replace('ROSL2A0N07', 'ROSLAN 2007')
            row[0]=row[0].replace('A ABDELSALAM2005', 'A ABDELSALAM 2005')
            row[0]=row[0].replace('AL-H20A0JR6I', 'AL-HAJRI 2006')
            row[0]=row[0].replace('Abhish2e0k0a8', 'Abhisheka 2008')
            row[0]=row[0].replace('Al2 M00E8QBAALI', 'Al MEQBAALI 2008')                        
                                            
            if 'CHAMPIONSHIPS' in row[0]:
            
                competition=row[0]
                                                
            if 'MEN' in row[0]:
                gender='Male'
                
            if 'WOMEN' in row[0]:
                gender='Female'    
                
            
            # Extract event and stage   
                        
            if re.search(event_pattern, row[0]): # isolate event in two steps
                                
                epos = re.search(event_pattern, row[0])  # find row
                
                start = epos.end()
                
                string = row[0][start:] # strip out event part
                
                
                                
                try:
                    spos = re.search(stage_pattern, string)  # check if row contains stage indication
                    split_end = spos.start()-1
                    
                    event = string[:split_end]   # strip out stage
                    
                    stage = string[split_end:]
                    
                    event = event.strip()
                    
                    print(event)
                                        
                except:
                    
                    event = string.strip()    
                    
                    print(event)
                                        
                
                                 
                    
            # Extract Heat
            
            if 'HEAT' in row[0]:
                                
                heat=re.findall(heat_pattern, row[0])
                
                
            # Extract group (for long jumps)
            
            if 'GROUP' in row[0]:
                
                group=row[0]
                                                                    
                
            # Extract columns of table    
            
            if 'Pos' in row[0]: # extract column names
                            
                row_index=index  # where the header row is
                columns=df.iloc[row_index,0]
                column_names=columns.split(' ')
                                
                column_names.remove('First')
                column_names.remove('LAST')
                column_names.remove('Name,')   
            

             
            # Extract alpha 3 country code. Look for it after DOB
            
            
            try:
                                            
                find = re.search(DOB_pattern, row[0])
                            
                start = find.end()
                end = find.end()+3
            
                country = row[0][start:end].strip()
                                            
                if country in list_countries:
                       
                    pass
                
                else:
                    
                    country = None
                
                
            except:
                
                pass
                        
            
            # Extract TAG ID
            
            res = re.findall(tag_pattern, row[0])
            
            if len(res)>1:
                            
                tag_id = res[0]
                
                
            else:
                
                tag_id = res
                        
                        
            # Extract DOB
            
            dob = re.findall(DOB_pattern, row[0])
                                
                                    
            
            # Extract name (will pick up nation code as a 3rd element in list)
                        
          #  if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                
            
            if re.search(DOB_pattern, row[0]) and re.search(results_begin, row[0]):  # if line begins with a digit and dob pattern exists then a results line
                                                                  
                result = re.findall(name_pattern, row[0])   # does not catch some names corrupted at the end
                                              
                full_name = result[0]  
                                  
                
                                                            
              
            # Extract reaction time
            
            if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                            
            
                reaction = re.findall(react_time, row[0])
                
                
                
                            
                
            # Extract wind
            
            if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                            
            
                wind = re.findall(wind_pattern, row[0])
                
                        
                

            # Extract result. Put into a list, search for country code index and choose index after
            # Look for existance of tag_id, dob to find valid line
            
            
            if dob and tag_id:  # lines with DQ and DNS are not picking up because tag_id for those has no space in front
                
                
                result_list = row[0].split(' ')
                                
                
                try:
                
                    result_index = result_list.index(country)
                                        
                    result = result_list[result_index+1]
                                                                                
                                                            
                except:
                    
                    pass

                                        
                
            # Extract points. If 'points' is present in list of columns extract it from last element in list
                        
            if dob and tag_id and 'Points' in column_names and 'DNS' not in row[0]:
                
                
                list=row[0].split(' ')
                                
                points = list[-1]  
                
                rank = list[0]
                
                
            # Extract rank
                                
            if dob and tag_id and 'Pos.' in column_names:
                
                if re.search(rank_pattern, row[0]):  # check if line begins with a single digit = rank
            
                    list=row[0].split(' ')
                                                
                    rank = list[0]   # first item in list is the rank
                                                        
                else:
                    
                    rank = None  # competitor has no rank e.g. DNS, DQ etc.
                    
            # Extract Q/q and PB/SB
            
            
            if tag_id and dob:  # if an actual competitor            
                        
                values = result_list[result_index+2:]  # skip country ID and result for table of attempts
                                                    
                    
                try:
                                        
                    if values[-1] =='SB' or values[-1]=='PB':  # extract PB/SB
                        
                        remarks = values[-1]
                        
                        values.pop()  
                        
                        
                except:
                        
                    pass
                    
                    
                try:
                        
                    if values[0] == 'Q' or values[0] == 'q': # extract Q/q
                        
                        qualification = values[0]
                        
                        values.pop(0)
                                                    
                except:
                        
                    pass
                
    
               
            # append result into dataframe
        
        
            if event == '5000m' or event=='100m' or event=='Heptathlon 200m' or event=='200m' or event=='Decathlon 400m' or event=='400m' or event=='Heptathlon 800m' or event=='800m' or event=='1500m' or event=='Decathlon 1500m' or event=='3000m' or event=='3000m Steeplechase' or event=='Decathlon 110m Hurdles' or event=='10000m Race Walk':
                
                temp_list.append(event)
                temp_list.append(competition)
                temp_list.append(rank)
                temp_list.append(tag_id)
                temp_list.append(full_name)
                temp_list.append(dob)
                temp_list.append(country)
                temp_list.append(result)
                temp_list.append(remarks)
                temp_list.append(gender)
                temp_list.append(points)
                temp_list.append(qualification)
                temp_list.append(wind)
                temp_list.append(reaction)
                temp_list.append(heat)
                temp_list.append(' ')
                
       #         print('temp list', temp_list)
                    
                                    
                                
                # append to dataframe    
                    
                df_results.loc[len(df_results)] = temp_list

        
                    
                print('df entry', event, competition, rank, tag_id, full_name, dob, country, wind, reaction, result, group, dict_result, points, qualification, heat, stage, gender, remarks)
                    
    
                    
        #temp=pd.DataFrame(table)
        
            # reset variables for next loop
                            
            points = None
            stage=None
        #    reaction = None
            wind = None
            country = None
        #    dob = None
            full_name = None
            tag_id = None
         #   rank = None
         #   result = None
         #   heat = None
            res_dict = None
            remarks = None
            group = None
            qualification = None
            name_list = []
        
            list_of_list = []
            list_dict=[]
            
            temp_list=[]
            
            
     #   relay_names = []
        
        rank=None
        dob=None
        dict_result = None
        event = None
        result = None
        reaction = None
        heat = None
        
        
                                    
                
 #       temp.iloc[2:,:]  # drop first 2 rows

 #       temp['EVENT'] = final_match
 #       temp['GENDER'] = gender
 #       temp['DIVISION'] = div
 #       temp['STAGE'] = stage
 #       temp['COMPETITION'] = 'NSG'
 #       temp['REGION'] = 'Local'
 #       temp['CATEGORY_EVENT'] = category_event
 #       temp['test'] = 'H'+temp[5].map(str) # create a new temp field to map across wind
 #       temp['SOURCE'] = 'PDF'
        
 #       temp['DATE'] = '2024'
 #       temp['QUALIFICATION']=''
 #       temp['POINTS']=''
 #       temp['UNIQUE_ID']=''
 #       temp['COUNTRY']=''
 #       temp['DICT_RESULTS']=''
 #       temp['DOB']=''
 #       temp['GROUP']=''
 #       temp['ATHLETE_ID']=''
 #       temp['REACT_TIME']=''
 #       temp['FREE_FIELD']=''
 #       temp['FREE_FIELD2']=''
 #       temp['FREE_FIELD3']=''

        
  #      temp['WIND']=temp['test'].map(wind_dict) # map temp field across to wind data
        
  #      temp = temp[temp[0].str.contains("~~~ End of Listing ~~~") == False]  # drop useless rows
  #      temp = temp[temp[0].str.contains("W/Gauge :") == False]
  #      temp = temp[temp[0].str.contains("Pos") == False]
     #   temp = temp[temp[7].str.contains("DNS|NM|DQ|DNF") == False] # drop rows without results
        
                     
        


#df_table.rename(
#    columns={0: "RANK", 1: "Tag", 2: "NAME", 3: "TEAM", 4: "RESULT", 5: "HEAT", 6: "LANE", 7: "REMARKS"},
#    inplace=True,
#)
        
#df_table.drop('test', axis=1, inplace=True)    # drop temp columns used to map wind data   

#df_table = df_table.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
#                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
#                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'])


#df_table.to_csv('sectrack_result_22a.csv', sep=',', index=False, encoding='utf-8')
        

            
                       
                       

        
            
                


Decathlon - 100m
Discus Throw
100m
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None  ROUND 1 Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 1 492   Morimoto MISAKI  [' 2005 '] JPN ['-1.4'] ['0.168'] 12.20 None None None Q ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2 412   Nuoxin CHEN  [' 2008 '] CHN ['-1.4'] ['0.257'] 12.35 None None None Q ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 3 425   Cheuk Man Serena TANG  [' 2005 '] HKG ['-1.4'] ['0.163'] 12.41 None None None q ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 4 546   Izza

400m
df entry 400m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None  ROUND 1 Female None
df entry 400m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 1'] None Female None
df entry 400m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 1'] None Female None
df entry 400m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 1 603   Hukmova JONBIBI  [' 2005 '] UZB [] ['0.173'] 55.23 None None None Q ['HEAT 1'] None Female None
df entry 400m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2 414   Yalun WANG  [' 2005 '] CHN [] ['0.416'] 56.30 None None None Q ['HEAT 1'] None Female None
df entry 400m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 3 513   Kidenko SOFYA  [' 2007 '] KAZ [] ['0.151'] 56.39 None None None q ['HEAT 1'] None Female None
df entry 400m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 4 431   Jane Christa Ming Suet KARLSSO [' 2007 '] HKG [] ['0.139'] 56

Decathlon High Jump
400m Hurdles
400m Hurdles
Javelin Throw
100m
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None  ROUND 1 Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 1 492   Morimoto MISAKI  [' 2005 '] JPN ['-1.4'] ['0.168'] 12.20 None None None Q ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2 412   Nuoxin CHEN  [' 2008 '] CHN ['-1.4'] ['0.257'] 12.35 None None None Q ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 3 425   Cheuk Man Serena TANG  [' 2005 '] HKG ['-1.4'] ['0.163'] 12.41 None None None q ['HEAT 1'] None Female None
df entry 100m 21ST ASIAN U20 ATHLET

200m
df entry 200m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None  ROUND 1 Male None
df entry 200m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 4'] None Male None
df entry 200m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 4'] None Male None
df entry 200m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 1 355   Wirayut DAENKHANOB  [' 2007 '] THA ['+0.6'] ['0.164'] 20.99 None None None Q ['HEAT 4'] None Male PB
df entry 200m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2 280   Pengiran Aidil Auf Bin HAJAM  [' 2005 '] MAS ['+0.6'] ['0.150'] 21.61 None None None Q ['HEAT 4'] None Male None
df entry 200m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 3 118   King Wai YIP  [' 2005 '] HKG ['+0.6'] ['0.340'] 21.939 None None None Q ['HEAT 4'] None Male None
df entry 200m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 4 216   Egor POBEZHIMOV  [' 2007 '] KGZ ['+0

110m Hurdles
Decathlon Discus Throw
800m
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None  ROUND 1 Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 1'] None Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None ['HEAT 1'] None Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 1 187   Ochiai KO  [' 2006 '] JPN [] [] 01:51.33 None None None Q ['HEAT 1'] None Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2 301   Ibrahim Haydar H ABDALLA  [' 2006 '] QAT [] [] 01:52.01 None None None Q ['HEAT 1'] None Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 3 170   Mohammad Hesam MOZAFARI  [' 2005 '] IRI [] [] 01:52.72 None None None Q ['HEAT 1'] None Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 4 259   Faraj ALAJEEL  [' 2005 '] KUW [

3000m Steeplechase
df entry 3000m Steeplechase 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None  FINAL Female None
df entry 3000m Steeplechase 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None None Female None
df entry 3000m Steeplechase 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 1 450   Ekta DEY  [' 2005 '] IND [] [] 10:31.95 None None None None None None Female None
df entry 3000m Steeplechase 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2 600   Dilshoda USMONOVA  [' 2005 '] UZB [] [] 10:41.37 None None None None None None Female None
df entry 3000m Steeplechase 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 3 610   Soibova MASHHURA  [' 2008 '] UZB [] [] 11:31.35 None None None None None None Female None
df entry 3000m Steeplechase 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 4 557   Ana Marie EUGENIO  [' 2005 '] PHI [] [] 12:09.86 None None None None None None Female None
df entry 3000m Steeplechase 21ST AS

100m
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None  FINAL Male None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None None Male None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 1 109   Keli ZENG  [' 2006 '] CHN ['+0.4'] ['0.146'] 10.19 None None None None None None Male PB
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2 127   Yat Lok CHAN  [' 2007 '] HKG ['+0.4'] ['0.149'] 10.41 None None None None None None Male None
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 3 117   Chun Ting KWOK  [' 2007 '] HKG ['+0.4'] ['0.137'] 10.46 None None None None None None Male PB
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 4 265   Yaqoub ALAZEMI  [' 2005 '] KUW ['+0.4'] ['0.128'] 10.53 None None None None None None Male PB
df entry 100m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 5 341   Wijesinghe Merone JULIAN  [' 2005 '] SRI ['+0.4'] ['0.538',

Heptathlon - Long Jump
High Jump
100m Hurdles
800m
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None  FINAL Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS None [] None [] None None None None None None None None None None Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 1 187   Ochiai KO  [' 2006 '] JPN [] [] 01:48.01 None None None None None None Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2 300   Hatim AITOULGHAZI  [' 2006 '] QAT [] [] 01:48.39 None None None None None None Male PB
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 3 190   Tachizako HIRONORI  [' 2005 '] JPN [] [] 01:49.73 None None None None None None Male None
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 4 301   Ibrahim Haydar H ABDALLA  [' 2006 '] QAT [] [] 01:50.14 None None None None None None Male PB
df entry 800m 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 5 170   Mohammad Hesam MOZAFARI  [' 2005

In [893]:
df_results

,EVENT,COMPETITION,RANK,TAG_ID,NAME,DOB,COUNTRY,RESULT,REMARKS,GENDER,POINTS,QUALIFICATION,WIND,REACTION,HEAT,DICT_RESULT
0,100m,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,None,[],None,[],None,None,None,Female,None,None,None,None,None,
1,100m,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,None,[],None,[],None,None,None,Female,None,None,None,None,[HEAT 1],
2,100m,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,None,[],None,[],None,None,None,Female,None,None,None,None,[HEAT 1],
3,100m,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,1,492,Morimoto MISAKI,[ 2005 ],JPN,12.20,None,Female,None,Q,[-1.4],[0.168],[HEAT 1],
4,100m,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,2,412,Nuoxin CHEN,[ 2008 ],CHN,12.35,None,Female,None,Q,[-1.4],[0.257],[HEAT 1],
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,Heptathlon 800m,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,4,476,Elnaz MOUSAEI,[ 2005 ],IRI,02:27.18,None,Female,729,None,[],[],None,
517,Heptathlon 800m,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,5,599,Chikalova ANNA,[ 2006 ],UZB,02:31.60,None,Female,673,None,[],[],None,
518,Heptathlon 800m,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,6,459,Pavana NAGARAJ,[ 2005 ],IND,02:36.42,None,Female,615,None,[],[],None,
519,Heptathlon 800m,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,7,541,Nour Badr ALJANDAN,[ 2007 ],KSA,02:42.73,None,Female,542,None,[],[],None,


In [464]:
import pdfplumber
import os


os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/U20/') 

#file="sectrack_audit_2010_211-211.pdf"
#file="Session 13_13-end.pdf"
#file="Session 13_1-4.pdf"
#file="sectrack_audit_2010_28-28.pdf"
#file="Session 13_5-6.pdf"
file='All Results.pdf'   # stranded pdf



#table_settings = {
#    "vertical_strategy": "text",
#    "horizontal_strategy": "text",
#    }


with pdfplumber.open(file) as pdf:
    
    page = pdf.pages[6]
    table=page.extract_table()
    text=page.extract_text()


splitted=text.splitlines()

splitted



['21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS',
 'DUBAI, 24TH TO 27TH APRIL 2024',
 'Result Published at: 24/04/2024 , 9:30 AM',
 'WOMEN U20 -100m ROUND 1',
 'HEAT 1',
 'Pos. BIB First Name, LAST NAME Born Nation Result Wind React Time',
 '1 492 Morimoto MISAKI 2005 JPN 12.20 Q -1.4m/s 0.168',
 '2 412 Nuoxin CHEN 2008 CHN 12.35 Q -1.4m/s 0.257',
 '3 425 Cheuk Man Serena TANG 2005 HKG 12.41 q -1.4m/s 0.163',
 '4 546 Izzatul Musfirah Binti Ahmad Kama2l 0A0Z8IRA MAS 12.73 -1.4m/s 0.236',
 '5 444 Shava Salvia WARANGGANI 2008 INA 12.79 -1.4m/s 0.180',
 'HEAT 2',
 'Pos. BIB First Name, LAST NAME Born Nation Result Wind React Time',
 '1 419 Yujie CHEN 2008 CHN 11.76 Q -0.5m/s 0.179',
 '2 613 Tran THỊ 2005 VIE 11.78 Q -0.5m/s 0.181',
 '3 579 Jirapat KHANONTA 2005 THA 12.35 q -0.5m/s 0.253',
 '4 547 Ormilla Octavia Codday NYADAU 2006 MAS 12.46 -0.5m/s 0.220',
 '5 526 Yermola KRISTINA 2005 KAZ 12.48 -0.5m/s 0.201',
 '6 604 Karina KARIMOVA 2008 UZB 12.85 -0.5m/s 0.212',
 'HEAT 3',
 'Pos. BIB First Nam

In [341]:
table

[['HEAT 2'],
 ['Pos. BIB First Name, LAST NAME Born Nation Result Wind React Time'],
 ['1 419 Yujie CHEN 2008 CHN 11.76 Q -0.5m/s 0.179'],
 ['2 613 Tran THỊ 2005 VIE 11.78 Q -0.5m/s 0.181'],
 ['3 579 Jirapat KHANONTA 2005 THA 12.35 q -0.5m/s 0.253'],
 ['4 547 Ormilla Octavia Codday NYADAU 2006 MAS 12.46 -0.5m/s 0.220'],
 ['5 526 Yermola KRISTINA 2005 KAZ 12.48 -0.5m/s 0.201'],
 ['6 604 Karina KARIMOVA 2008 UZB 12.85 -0.5m/s 0.212'],
 [''],
 ['']]

In [25]:
pd.DataFrame(splitted)

,0
0,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
1,"DUBAI, 24TH TO 27TH APRIL 2024"
2,"Result Published at: 22/04/2024 , 7:06 PM"
3,WOMEN U20 - High Jump FINAL
4,"Pos. BIB First Name, LAST NAME Born Nation Result"
5,1 598 Barnokhon SAYFULLAEVA 2005 UZB 1.83
6,2 521 Meirmanova FAINA 2006 KAZ 1.76 PB
7,3 601 Emiliya RUDINA 2006 UZB 1.74 PB
8,4 518 Malyugina ARINA 2007 KAZ 1.74
9,5 439 Zhichu YANG 2007 HKG 1.65


In [382]:
event_pattern='(MEN U20 -)|(WOMEN U20 -)'
stage_pattern='FINAL|QUALIFYING ROUND|ROUND 1|SEMI FINAL'
#name_pattern='[a-zA-Z]{3,20}'
#name_pattern='[A-Z]{1}[a-z]{2,10}'
#name_pattern='\b[A-Z][a-z]{2-10}[A-Z]\b'
mixed_up = '[a-zA-Z0-9]{5,}'

country_code = '\s[A-Z]{3}\s'
DOB_pattern='20[0-9]{2}\s'
tag_pattern='\s[0-9]{3}\s'


#text = 'WOMEN U20 - High Jump FINAL'
#text = '1 429 Hoi Man Chloe PAK 2006 HKG 22.23 Q -1.1m/s 0.156'
# text = '4 546 Izzatul Musfirah Binti Ahmad Kama2l 0A0Z8IRA MAS 12.73 -1.4m/s 0.236'
# text ='4 288 Al Mulham Said Al Abd Obaid AL SH2E0K0I5LI OMA 10.92 +2.0m/s 0.171 PB'
#text = '337 Rajapaksha Pathiranage Don Avishka T3h3e4mindaJ aRyAaJtAhPuAnKgSaHlaAge Omel Shashintha SILVA'
# text = '568 Ambepitiya Wijethunga Gamage Jithm5a6 S9henarDi WunIuJEsTuHngUhNeG MAudiyanselage Maheema Pravindi DUNUSINGHE'
# text = '338 Uyangoda Hewage Hasindu NETHSARA333 Gonsalu Wasan Jathya KIRULU'
# text = '339 Warnakulasooriya Malith Koushan THA3M40EL Weeraratna Dineth INDUWARA'
# text = '278 Muhammad Sayyid Amin Bin ROSLAN274 Dexter BEDIE'
# text = '4 332 Edirisingha Siriwardanalage Denet2h0 A0N5UHAS SRI 2.06'
# text = '5 330 Agampodi Tharusha Dinushan MEN20D0IS8 SRI 2.06'
# text = '337 Rajapaksha Pathiranage Don Avishka T5h7e1mindaK RoAdiJtAhPuAwKaSkHkAu Takshima NUHANSA'
# text = '10 290 Hajan Abdullah Yousuf Ahmed AL-H20A0JR6I OMA 56.04 X X 56.04'
# text = '11 371 Nasser Saeed Nasser Saeed ALKATH20E0E6RI UAE 54.21 54.21 50.54 48.34 PB'
# text = '4 572 Mahadura Gedara Tharushi Abhish2e0k0a8 PEMASIRISRI 02:10.52'
# text = '1 314 Seifeldin Mohamed A ABDELSALAM2005 QAT 5.51'
# text = '6 335 Kuda Liyanage Ayomal AKALANKA2006 SRI 52.84 0.251'
# text = '1 481 Nazanin Fatemeh EIDIYANKAKHAK2I005 IRI 58.86 0.239 PB'
# text = '8 373 Salem Yousufsalem Mohammad Al2 M00E8QBAALI UAE 12.76 12.73 X 12.76 X X 12.35'
# text ='7 367 Hasan Taleb Hasan Ahmed ALDHUH20O0O7RI UAE 11.43 -0.8m/s 0.182'
# text = '5 309 Mohammed Ghanim M A AL-KUWA20R0I7 QAT 11.03 +1.0m/s 0.140 PB'
# text = '5 309 Mohammed Ghanim M A AL-KUWA20R0I7 QAT 11.03 +1.0m/s 0.140 PB'
# text = '3 171 Seyed Mohammad Sajjad AGHAEI2005 IRI 47.99 q 0.241'
text = '4 431 Jane Christa Ming Suet KARLSSON2007 HKG 56.98 q 0.139'
#text = '1 314 Seifeldin Mohamed A ABDELSALAM2005 QAT 5.51'

# text = '10 290 Hajan Abdullah Yousuf Ahmed AL-H20A0JR6I OMA 56.04 X X 56.04'
# text = '11 371 Nasser Saeed Nasser Saeed ALKATH20E0E6RI UAE 54.21 54.21 50.54 48.34 PB'


#text = '5 552 Ala"a Abdulla Mohamed AL-ZADJA2LI005 OMA 22.69 -1.1m/s 0.212'

result = re.findall(mixed_up, text)  


In [383]:
result

['Christa', 'KARLSSON2007']

In [292]:
text_list=text.split(' ')

In [291]:
print(re.sub("[a-z] [A-Z]", ".", text))

1 429 Ho.a.hlo.AK 2006 HKG 22.23 Q -1.1m/s 0.156


In [293]:
idx = [i for i, item in enumerate(text_list) if re.search(name_pattern, item)]


In [294]:
idx

[2, 3, 4]

In [910]:
# Scrape U20 2024 PDFs
# For extracting triple jump and long jump format results only

# TODO : correct data for first pole vault event, correct algo for picking up and correcting corrupted names

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/U20/All Results.pdf"

 
# Iterate over files in directory

event_pattern ='(MEN U20 -)|(WOMEN U20 -)'

heat_pattern = 'HEAT \d'

school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
year_pattern='\d\d\d\d'

#name_pattern='\d\d\d.{10-29}\d\d\d\d'

stage_pattern='FINAL|QUALIFYING|ROUND|SEMI|GROUP'

country_code = '\s[A-Z]{3}\s'

DOB_pattern='\s20[0-9]{2}\s'

#tag_pattern='\s[0-9]{3}\s'

tag_pattern='[0-9]{3}\s'

last_name_pattern = '[A-Z]{2,}'

wind_pattern='(\+\d\.\d|\-\d\.\d)'

name_pattern='[a-zA-Z\-\. \"]{3,31}'

results_begin = '^\d'  # line begins with a digit

react_time = '0\.[0-9]{3}'

rank_pattern = '^\d |^\dd '

qualification_pattern = ' Q | q '

relay_pattern = '\d\d\:\d\d\.\d\d'

corrupted_names = '[A-Z0-9]{5,}'

timing_pattern = '\d\d\.\d\d|\d.\d\d'




df_table=pd.DataFrame()  # initialize empty master df

df=None

temp=pd.DataFrame(columns = ['Pos.', 'BIB', 'NAME', 'Born', 'Nation', 'Result', 'Wind', 'React', 'Time', 'Points'])

df_results=pd.DataFrame(columns = ['EVENT', 'COMPETITION', 'RANK', 'TAG_ID', 'NAME', 'DOB', 'COUNTRY', 'RESULT', 'REMARKS', 'GENDER', 'POINTS', 'QUALIFICATION', 'WIND', 'REACTION', 'HEAT', 'DICT_RESULT'])


with pdfplumber.open(file) as pdf:
        
        
    for i in range(len(pdf.pages)):    
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()        
        
        df=pd.DataFrame(text.splitlines())
        
                        
        for j in range(len(df)): # find row containing event details and column names  
            
            # Look for corrupted names and correct
            
            corrupted = re.findall(corrupted_names, df[0][j])
            
        
         #   print('corrupted', j, corrupted, df[0][j])
            
            df.loc[j, 0] = df.loc[j, 0].replace('M2a0d0u5shani HERSARTIH', 'Madushani HERSARTIH 2005 SRI')
            df.loc[j, 0] = df.loc[j, 0].replace('Kama2l 0A0Z8IRA', 'Kamal AZIRA 2008')
            df.loc[j, 0] = df.loc[j, 0].replace('AL SH2E0K0I5LI', 'AL SHEKILI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('ALDHUH20O0O7RI', 'ALDHUHOORI 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('KUWA20R0I7', 'KUWARI 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('KARLSSON2007', 'KARLSSON 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('EIDIYANKAKHAK2I005', 'EIDIYANKAKHAKI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('AGHAEI2005', 'AGHEI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('SWAKAR2007', 'SWAKAR 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('AKALANKA2006', 'AKALANKA 2006')
            df.loc[j, 0] = df.loc[j, 0].replace('ZADJA2LI005', 'ZADJALI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('ROSL2A0N07', 'ROSLAN 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('A ABDELSALAM2005', 'A ABDELSALAM 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('AL-H20A0JR6I', 'AL-HAJRI 2006')
            df.loc[j, 0] = df.loc[j, 0].replace('Abhish2e0k0a8', 'Abhisheka 2008')
            df.loc[j, 0] = df.loc[j, 0].replace('Al2 M00E8QBAALI', 'Al MEQBAALI 2008')
            df.loc[j, 0] = df.loc[j, 0].replace('ALSHANQ2IT0I07', 'ALSHANQITI 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('PRAMUDIK5A71', 'PRAMUDIKA')
            df.loc[j, 0] = df.loc[j, 0].replace('Jithm5a6 S9henarDi WunIuJEsTuHngUhNeG', 'Jithma Shenari Wunusunghe')
            df.loc[j, 0] = df.loc[j, 0].replace('THA3M40EL', 'THAMEL')
            df.loc[j, 0] = df.loc[j, 0].replace('T5h7e1mindaK RoAdiJtAhPuAwKaSkHkAu', 'Theminda Rodithuwakku')
            df.loc[j, 0] = df.loc[j, 0].replace('ALKATH20E0E6RI', 'ALKATHEERI 2006')
            df.loc[j, 0] = df.loc[j, 0].replace('Ahme2d0 0A7LMESSABIUAE', 'Ahmed ALMESSABIUAE 2007')

            



            #df[0][j]=df[0][j].replace('M2a0d0u5shani HERSARTIH', 'Madushani HERSARTIH 2005 SRI')
            #df[0][j]=df[0][j].replace('Kama2l 0A0Z8IRA', 'Kamal AZIRA 2008')
           # df[0][j]=df[0][j].replace('AL SH2E0K0I5LI', 'AL SHEKILI 2005')
            #df[0][j]=df[0][j].replace('ALDHUH20O0O7RI', 'ALDHUHOORI 2007')
           # df[0][j]=df[0][j].replace('KUWA20R0I7', 'KUWARI 2007')
          #  df[0][j]=df[0][j].replace('KARLSSON2007', 'KARLSSON 2007')
         #   df[0][j]=df[0][j].replace('EIDIYANKAKHAK2I005', 'EIDIYANKAKHAKI 2005')
          #  df[0][j]=df[0][j].replace('AGHAEI2005', 'AGHEI 2005')
          #  df[0][j]=df[0][j].replace('SWAKAR2007', 'SWAKAR 2007')
          #  df[0][j]=df[0][j].replace('AKALANKA2006', 'AKALANKA 2006')
          #  df[0][j]=df[0][j].replace('ZADJA2LI005', 'ZADJALI 2005')
          #  df[0][j]=df[0][j].replace('ROSL2A0N07', 'ROSLAN 2007')
          #  df[0][j]=df[0][j].replace('A ABDELSALAM2005', 'A ABDELSALAM 2005')
          #  df[0][j]=df[0][j].replace('AL-H20A0JR6I', 'AL-HAJRI 2006')
           # df[0][j]=df[0][j].replace('Abhish2e0k0a8', 'Abhisheka 2008')
          #  df[0][j]=df[0][j].replace('Al2 M00E8QBAALI', 'Al MEQBAALI 2008')   
                        
            if 'CHAMPIONSHIPS' in df[0][j]:
                competition=df[0][j]
                
            print('event', event)
                                                
            if 'MEN' in df[0][j]:
                gender='Male'
                
            if 'WOMEN' in df[0][j]:
                gender='Female'   
                
                    
            if re.search(event_pattern, df[0][j]): # isolate event in two steps
                                
                epos = re.search(event_pattern, df[0][j])  # find row
                
                start = epos.end()
                
                string = df[0][j][start:] # strip out event part
                
                                
                try:
                    spos = re.search(stage_pattern, string)  # check if row contains stage indication
                    split_end = spos.start()-1
                    
                    event = string[:split_end]   # strip out stage
                    
                    stage = string[split_end:]
                    
                    event = event.strip()
                    
                    print(event)
                                        
                except:
                    
                    event = string.strip()    
                    
                    print(event)
                    
             # Extract Heat
            
            if 'HEAT' in df[0][j]:
                                
                heat=re.findall(heat_pattern, df[0][j])
                                
            # Extract group (for long jumps)
            
            if 'GROUP' in df[0][j]:
                
                group=df[0][j]
                
            # Extract columns of table    
            
            if 'Pos' in df[0][j]: # extract column names
                            
                row_index=index  # where the header row is
                columns=df[0][j]
                column_names=columns.split(' ')
                                
                column_names.remove('First')
                column_names.remove('LAST')
                column_names.remove('Name,')            

           
                

            if event=='Long Jump' or event == 'Triple Jump':   
                
                print(df[0][j])
                
                
                
                
            # Extract alpha 3 country code. Look for it after DOB
            
            
                try:
                                            
                    find = re.search(DOB_pattern, df[0][j])
                                
                    start = find.end()
                    end = find.end()+3
            
                    country = df[0][j][start:end].strip()
                                                            
                    if country in list_countries:
                       
                        pass
                
                    else:
                    
                        country = None
                
                
                except:
                
                    pass
                
                print('country', country)
                
                
            
            # Extract TAG ID
            
                res = re.findall(tag_pattern, df[0][j])
                            
                if len(res)>1:
                            
                    tag_id = res[0]
                
                
                else:
                
                    tag_id = res
                    
                print('tag id', tag_id)
                                
            
                        
            # Extract DOB
            
                dob = re.findall(DOB_pattern, df[0][j])
                
                print('dob', dob)
                                        
                                    
            
            # Extract name (will pick up nation code as a 3rd element in list)
                        
          #  if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                
            
                if re.search(DOB_pattern, df[0][j]) and re.search(results_begin, df[0][j]):  # if line begins with a digit and dob pattern exists then a results line
                                                                  
                    result = re.findall(name_pattern, df[0][j])   # does not catch some names corrupted at the end
                    
                                              
                    full_name = result[0]  
                                    
                    if event=='Pole Vault':  # if vault event put names into a dictionary
                    

                        name_list.append(full_name)
                                                            
              
            # Extract reaction time
            
                if re.findall(DOB_pattern, df[0][j]) and re.search(results_begin, df[0][j]):
                            
            
                    reaction = re.findall(react_time, df[0][j])
                
                print('reaction', reaction)
                
                         
            # Extract wind
            
                if re.findall(DOB_pattern, df[0][j]) and re.search(results_begin, df[0][j]):
                            
            
                    wind = re.findall(wind_pattern, df[0][j])
                
                print('wind', wind)
                                               

            # Extract result. Put into a list, search for country code index and choose index after
            # Look for existance of tag_id, dob to find valid line
            
            
                if re.search(DOB_pattern, df[0][j]) and re.search(tag_pattern, df[0][j]):  # lines with DQ and DNS are not picking up because tag_id for those has no space in front
                
                
                    result_list = df[0][j].split(' ')
                                                    
                
                    try:
                
                        result_index = result_list.index(country)
                                        
                        result = result_list[result_index+1]
                                                                                
                                                            
                    except:
                    
                        pass
                    
                print('result', result)
                    
                                                        
                
            # Extract points. If 'points' is present in list of columns extract it from last element in list
                        
                if re.search(DOB_pattern, df[0][j]) and re.search(tag_pattern, df[0][j]) and 'Points' in column_names and 'DNS' not in df[0][j]:
                
                
                    list=df[0][i].split(' ')
                                
                    points = list[-1]  
                
                    rank = list[0][j]
                    
                
                
            # Extract rank
                                
                if re.search(DOB_pattern, df[0][j]) and re.search(tag_pattern, df[0][j]) and 'Pos.' in column_names:
                    
                    
                    if re.search(rank_pattern, df[0][j]):  # check if line begins with a single digit = rank
            
                        list=df[0][j].split(' ')
                                                
                        rank = list[0]   # first item in list is the rank
                                                        
                    else:
                    
                        rank = None  # competitor has no rank e.g. DNS, DQ etc.
                        
                print('rank', rank)
                    
            # Extract Q/q and PB/SB
            
            
                if re.search(DOB_pattern, df[0][j]) and re.search(tag_pattern, df[0][j]):  # if an actual competitor            
                        
                    values = result_list[result_index+2:]  # skip country ID and result for table of attempts
                                                    
                    
                    try:
                                        
                        if values[-1] =='SB' or values[-1]=='PB':  # extract PB/SB
                        
                            remarks = values[-1]
                        
                            values.pop()  
                        
                        
                    except:
                        
                        pass
                    
                    
                    try:
                        
                        if values[0] == 'Q' or values[0] == 'q': # extract Q/q
                        
                            qualification = values[0]
                        
                            values.pop(0)
                                                    
                    except:
                        
                        pass
 
                
                
                    print('index', index)
                                                
                    print('row', df[0][j])
                                
                    idx = column_names.index('Result')
                
                    keys = column_names[idx+1:]  #  attempts come one after 'results' columns   
                
                    print('keys', keys)
                              
                
                    if dob and tag_id:  # if a results line
                    
                                                                                      
                        name_match = re.findall(name_pattern, df[0][j])   # extract name
                                              
                        full_name = name_match[0]  
                        
                        print('full name', full_name)
                
                
                                        
                        print('country', country)
                                        
                        country_idx = result_list.index(country)
                    
                                        
                        attempt_values = result_list[country_idx+2:]  # extract list of attempts
                    
                                        
        
    #                if re.search('DOB_pattern', str(values[0])):  # first values is DOB choose next values onwards
                                 
    #                    values = result_list[result_index+3:]
                                
    #                    print('new values', values)        
            
                                        
                                        
                        if attempt_values[-1] =='SB' or attempt_values[-1]=='PB':  # extract PB/SB
                        
                            remarks = attempt_values[-1]
                        
                            attempt_values.pop()  
                        
                            print('remarks', remarks)
                        
                        try:
                        
                            if attempt_values[0] == 'Q' or attempt_values[0] == 'q': # extract Q/q
                        
                                qualification = attempt_values[0]
                        
                                attempt_values.pop(0)
                            
                        except:
                        
                            pass
                                        
                    
                    
                    try:
                        

                        if 'm/s' in df[0][j+1]:
                
                            wind_list=df[0][j+1].split(' ')
                            wind_values=wind_list[1:]
                                        
                
                            wind = str(wind_list[0]).rstrip('m/s')
                    
                    except:
                        
                        pass
                
            
                # Compare attempt values and wind values lists. X attempt should have null wind value
                
                    for k in range(len(attempt_values)):
                        
                        
                        if attempt_values[k]=='X' and len(attempt_values)>len(wind_values):
                            wind_values.insert(k, ' ')
                            
                        else:
                            pass
                            
                
               
                 # zip wind and results lists into two values per key dictionary
                    print('name', full_name)

                    print('keys', keys)
                    print('winds', wind_values)
            
                    print('result', result)
    
                    dict_result = dict(zip(keys, zip(attempt_values, wind_values))) 
        
                    print('dict', dict_result)
            
            
            
                    temp_list.append(event)
                    temp_list.append(competition)
                    temp_list.append(rank)
                    temp_list.append(tag_id)
                    temp_list.append(full_name)
                    temp_list.append(dob)
                    temp_list.append(country)
                    temp_list.append(result)
                    temp_list.append(remarks)
                    temp_list.append(gender)
                    temp_list.append(points)
                    temp_list.append(qualification)
                    temp_list.append(wind)
                    temp_list.append(reaction)
                    temp_list.append(heat)
                    temp_list.append(dict_result)
                
                    print('temp list', temp_list)
                    
                                    
                                
                # append to dataframe
                
                if tag_id!=[] and full_name is not None and rank is not None and dob!=[] and event is not None and result is not None and dict_result is not None:
                    
                    df_results.loc[len(df_results)] = temp_list

        
                    
                print('df entry', event, rank, tag_id, full_name, dob, country, wind, reaction, result, group, dict_result, points, qualification, heat, stage, gender, remarks)
#
                temp_list=[]


    
                            
            # reset variables for next loop
            
                
            points = None
        #    reaction = None
            wind = None
            country = None
            dob = None
            full_name = None
            tag_id = None
            rank = None
         #   result = None
         #   heat = None
            res_dict = None
            remarks = None
            group = None
            qualification = None
            name_list = []
        
            list_of_list = []
            
            
            
     #   relay_names = []
        
        
        event = None
        result = None
        reaction = None
        heat = None
        dict_result = None

        
        

event Triple Jump
All Results
country UZB
tag id []
dob []
reaction []
wind +1.6
result 13.33
rank 1
df entry Triple Jump 1 []  Khushnoza SHAVKATOVA  [] UZB +1.6 [] 13.33 None {'1': ('13.02', '-0.0'), '2': ('13.20', '-0.0'), '3': ('13.33', '+1.6'), '4': ('12.97', '+1.2'), '5': ('X', '+0.6'), '6': ('X', '-1.0')} None None None  FINAL Female SB
event Triple Jump
24-27 April 2024
country None
tag id []
dob []
reaction []
wind None
result 13.33
rank None
df entry Triple Jump None [] None [] None None [] 13.33 None {'1': ('13.02', '-0.0'), '2': ('13.20', '-0.0'), '3': ('13.33', '+1.6'), '4': ('12.97', '+1.2'), '5': ('X', '+0.6'), '6': ('X', '-1.0')} None None None  FINAL Female None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event None
event

event None
event None
event None
event None
200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event None
event None
event None
200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event None
event None
event None
event None
3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event 3000m
event None
event None
event None
event None
1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500m
event 1500

event None
event None
event None
event None
Decathlon Discus Throw
event Decathlon Discus Throw
event Decathlon Discus Throw
event Decathlon Discus Throw
event Decathlon Discus Throw
event Decathlon Discus Throw
event Decathlon Discus Throw
event Decathlon Discus Throw
event None
event None
event None
event None
800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event 800m
event None
event None
event None
event None
Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event None
event None
event None
event None
200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
eve

event None
event None
event None
4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event None
event None
event None
event None
Heptathlon High Jump
event Heptathlon High Jump
event Heptathlon High Jump
event Heptathlon High Jump
event Heptathlon High Jump
event Heptathlon High Jump
event Heptathlon High Jump
event Heptathlon High Jump
event Heptathlon High Jump
event He

event None
event None
event None
event None
5000m
event 5000m
event 5000m
event 5000m
event 5000m
event 5000m
event 5000m
event 5000m
event 5000m
event 5000m
event None
event None
event None
4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event None
event None
event None
4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event 4x100m Relay
event None
event None
event None
4x400m Relay
event 4x400m Relay
event 4x400m Relay
e

In [911]:
df_results

,EVENT,COMPETITION,RANK,TAG_ID,NAME,DOB,COUNTRY,RESULT,REMARKS,GENDER,POINTS,QUALIFICATION,WIND,REACTION,HEAT,DICT_RESULT
0,Triple Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,1,605,Khushnoza SHAVKATOVA,[ 2005 ],UZB,13.33,SB,Female,None,None,+1.6,[],None,"{'1': ('13.02', '-0.0'), '2': ('13.20', '-0.0'..."
1,Triple Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,2,570,Herath Mudiyanselage Nethmika,[ 2005 ],SRI,13.01,None,Female,None,None,-0.2,[],None,"{'1': ('12.50', '-0.1'), '2': ('12.58', '-0.3'..."
2,Triple Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,3,533,Seongyi JANG,[ 2005 ],KOR,12.67,SB,Female,None,None,+0.1,[],None,"{'1': ('12.44', '-0.2'), '2': ('12.59', '-0.2'..."
3,Triple Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,4,441,Katyea Ebri SAFITRI,[ 2007 ],INA,12.39,PB,Female,None,None,+0.2,[],None,"{'1': ('X', '-0.0'), '2': ('12.28', '-0.0'), '..."
4,Triple Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,5,512,Khaibullina DILYARA,[ 2005 ],KAZ,12.20,None,Female,None,None,-0.1,[],None,"{'1': ('X', '-0.0'), '2': ('12.08', '-0.3'), '..."
5,Triple Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,6,462,Rishika AWASTHI,[ 2005 ],IND,12.13,PB,Female,None,None,+0.7,[],None,"{'1': ('12.01', '+2.1'), '2': ('X', '-0.0'), '..."
6,Triple Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,7,567,Zhong CHUHAN,[ 2005 ],SGP,12.10,SB,Female,None,None,+0.2,[],None,"{'1': ('X', '-0.0'), '2': ('12.10', '+0.2'), '..."
7,Triple Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,8,534,Seonyoo YUN,[ 2005 ],KOR,11.89,None,Female,None,None,+0.9,[],None,"{'1': ('11.66', '-0.0'), '2': ('X', '-0.0'), '..."
8,Triple Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,9,510,Ivankina ANASTASSIYA,[ 2006 ],KAZ,16.10,PB,Female,None,None,+0.2,[],None,"{'1': ('15.70', '+0.2'), '2': ('16.10', '+0.2'..."
9,Long Jump,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,1,146,Mohd Atta SAZID,[ 2006 ],IND,7.40,PB,Male,None,Q,+0.4,[],None,"{'1': ('7.40', '+0.4')}"


In [724]:
# Scrape U20 2024 PDFs
# Specific code for 4x400m relay event
# TODO : correct data for first pole vault event, correct algo for picking up and correcting corrupted names

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/U20/All Results.pdf"

 
# Iterate over files in directory

event_pattern ='(MEN U20 -)|(WOMEN U20 -)'

heat_pattern = 'HEAT \d'

school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
year_pattern='\d\d\d\d'

#name_pattern='\d\d\d.{10-29}\d\d\d\d'

stage_pattern='FINAL|QUALIFYING|ROUND 1|SEMI FINAL|GROUP'

country_code = '\s[A-Z]{3}\s'

DOB_pattern='\s20[0-9]{2}\s'

#tag_pattern='\s[0-9]{3}\s'

tag_pattern='[0-9]{3}\s'

last_name_pattern = '[A-Z]{2,}'

wind_pattern='(\+\d\.\d|\-\d\.\d)'

name_pattern='[a-zA-Z\-\. \"]{3,31}'

results_begin = '^\d'  # line begins with a digit

react_time = '0\.[0-9]{3}'

rank_pattern = '^\d |^\dd '

qualification_pattern = ' Q | q '

relay_pattern = '\d\d\:\d\d\.\d\d'

corrupted_names = '[A-Z0-9]{5,}'

timing_pattern = '\d\d\.\d\d|\d.\d\d'




df_table=pd.DataFrame()  # initialize empty master df

df=None

final_row =[]

temp=pd.DataFrame(columns = ['Pos.', 'BIB', 'NAME', 'Born', 'Nation', 'Result', 'Wind', 'React', 'Points', 'Dict_Result'])



with pdfplumber.open(file) as pdf:
        
        
    for i in range(len(pdf.pages)):    
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()        
        
        df=pd.DataFrame(text.splitlines())
        
                        
        for index, row in df.iterrows(): # find row containing event details and column names  
            
            # Look for corrupted names and correct
            
            #corrupted = re.findall(corrupted_names, row[0])
            
            #print('corrupted', index, corrupted)
            
            row[0]=row[0].replace('M2a0d0u5shani HERSARTIH', 'Madushani HERSARTIH 2005 SRI')
            row[0]=row[0].replace('Kama2l 0A0Z8IRA', 'Kamal AZIRA 2008')
            row[0]=row[0].replace('AL SH2E0K0I5LI', 'AL SHEKILI 2005')
            row[0]=row[0].replace('ALDHUH20O0O7RI', 'ALDHUHOORI 2007')
            row[0]=row[0].replace('KUWA20R0I7', 'KUWARI 2007')
            row[0]=row[0].replace('KARLSSON2007', 'KARLSSON 2007')
            row[0]=row[0].replace('EIDIYANKAKHAK2I005', 'EIDIYANKAKHAKI 2005')
            row[0]=row[0].replace('AGHAEI2005', 'AGHEI 2005')
            row[0]=row[0].replace('SWAKAR2007', 'SWAKAR 2007')
            row[0]=row[0].replace('AKALANKA2006', 'AKALANKA 2006')
            row[0]=row[0].replace('ZADJA2LI005', 'ZADJALI 2005')
            row[0]=row[0].replace('ROSL2A0N07', 'ROSLAN 2007')
            row[0]=row[0].replace('A ABDELSALAM2005', 'A ABDELSALAM 2005')
            row[0]=row[0].replace('AL-H20A0JR6I', 'AL-HAJRI 2006')
            row[0]=row[0].replace('Abhish2e0k0a8', 'Abhisheka 2008')
            row[0]=row[0].replace('Al2 M00E8QBAALI', 'Al MEQBAALI 2008')                        
                                            
            if 'CHAMPIONSHIPS' in row[0]:
            
                event=row[0]
                                                
            if 'MEN' in row[0]:
                gender='Male'
                
            if 'WOMEN' in row[0]:
                gender='Female'    
                
            
            # Extract event and stage   
                        
            if re.search(event_pattern, row[0]): # isolate event in two steps
                                
                epos = re.search(event_pattern, row[0])  # find row
                
                start = epos.end()
                
                string = row[0][start:] # strip out event part
                
                                
                try:
                    spos = re.search(stage_pattern, string)  # check if row contains stage indication
                    split_end = spos.start()-1
                    
                    event = string[:split_end]   # strip out stage
                    
               #     stage = string[split_end:]
            
                    stage = re.findall(stage_pattern, string)[0]
                                                        
                    event = event.strip()
                    
                    print(event)
                                        
                except:
                    
                    event = string.strip()    
                    
                    print(event)
                                        
                
                                 
                    
            # Extract Heat
            
            if 'HEAT' in row[0]:
                                
                heat=re.findall(heat_pattern, row[0])
                
            # Extract group (for long jumps)
            
            if 'GROUP' in row[0]:
                
                group=row[0]
                                                                    
                
        
             
            #-------------------------------------------------------
            # Event specific code below
            #-------------------------------------------------------
                     
            # Special code for vault events to capture table of attempts
            
            
            
           # Special code for Relays
           # Need to correct tag_id which should be None for relays
                        
            if event == '4x400m Relay':
                                
                # Extract alpha 3 country code. Look for it after DOB
                
                # Extract columns of table    
            
                if 'Pos' in row[0]: # extract column names
                            
                    row_index=index  # where the header row is
                    columns=df.iloc[row_index,0]
                    column_names=columns.split(' ')
                                
                    column_names.remove('First')
                    column_names.remove('LAST')
                    column_names.remove('Name,')            

        
                        
            # Extract DOB
            
                dob = re.findall(DOB_pattern, row[0])
                        
                                    
            
            # Extract name (will pick up nation code as a 3rd element in list)                        
            # if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                
            
                if re.search(DOB_pattern, row[0]) and re.search(results_begin, row[0]):  # if line begins with a digit and dob pattern exists then a results line
                                                                  
                    result = re.findall(name_pattern, row[0])   # does not catch some names corrupted at the end
                                              
                    full_name = result[0]  
                                                                                        
              
            # Extract reaction time
            
                if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                            
            
                    reaction = re.findall(react_time, row[0])
                
                
            # Extract wind
            
                if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                            
            
                    wind = re.findall(wind_pattern, row[0])
                
                        
            # Extract result. Put into a list, search for country code index and choose index after
            # Look for existance of tag_id, dob to find valid line
            
            
                if dob and tag_id:  # lines with DQ and DNS are not picking up because tag_id for those has no space in front
                
                
                    result_list = row[0].split(' ')
                                
                
                    try:
                
                        result_index = result_list.index(country)
                                        
                        result = result_list[result_index+1]
                                                                                
                                                            
                    except:
                    
                        pass
    
                                
                
            # Look for results line beginning with a rank
            # Extract rank, country, react time and result
                
                if re.search(name_pattern, row[0]) and re.search('^\d ', row[0]):
                    
                    relay_names =[]
                    
                    list = row[0].split(' ')                    
                    
                    rank = list[0]
                    country = re.findall(country_code, row[0])[0]
                    result = re.findall(relay_pattern, row[0])[0]
                                                              
                    final_element = list[-1]
                    
                    if re.search(react_time, final_element):  # extract react time
                        
                        reaction = final_element
                                                
                                        
            # Look for list of competitors and create dictionary
                
                if re.search(name_pattern, row[0]) and re.search(tag_pattern, row[0]):                    
                    
                    list_competitors = row[0]
                    
                    tags = re.findall(tag_pattern, row[0])
                                        
                    names = re.findall(name_pattern, row[0])
                    
                                        
                    dict_names = dict(zip(tags, names))
                    
                    relay_names.append(dict_names)
                    
                    
                if event is not None and country is not None and result is not None and rank is not None and len(relay_names)==2 and stage is not None and gender is not None:
                
                    print('output', event, rank, tag_id, relay_names, dob, country, wind, reaction, result, res_dict, points, qualification, heat, stage, gender, remarks)

            #        final_row.append(rank)
            #        final_row.append(tag_id)
            #        final_row.append(relay_names)
            #        final_row.append(dob)
             #       final_row.append(country)
            #        final_row.append(result)
            #        final_row.append(wind)
            #        final_row.append(reaction)
            #        final_row.append(points)
            #        final_row.append(' ')
                    
            #        print('final row', final_row)
                    
            #        df.loc[len(df)] = final_row



                    
                   
                
        
         #   print('reset 1')  
                
        #    points = None
        #    reaction = None
        #    wind = None
        #    country = None
        #    dob = None
        #    full_name = None
        #    tag_id = None
         #   rank = None
         #   result = None
         #   heat = None
         #   res_dict = None
         #   remarks = None
         #   group = None
         #   qualification = None
            name_list = []
        
            list_of_list = []
            list_dict=[]
            
        print('reset 2') 
     #   relay_names = []
        
        rank=None
        dob=None
        dict_result = None
        event = None
        result = None
        reaction = None
        heat = None
        
        
                                    
                
 #       temp.iloc[2:,:]  # drop first 2 rows

 #       temp['EVENT'] = final_match
 #       temp['GENDER'] = gender
 #       temp['DIVISION'] = div
 #       temp['STAGE'] = stage
 #       temp['COMPETITION'] = 'NSG'
 #       temp['REGION'] = 'Local'
 #       temp['CATEGORY_EVENT'] = category_event
 #       temp['test'] = 'H'+temp[5].map(str) # create a new temp field to map across wind
 #       temp['SOURCE'] = 'PDF'
        
 #       temp['DATE'] = '2024'
 #       temp['QUALIFICATION']=''
 #       temp['POINTS']=''
 #       temp['UNIQUE_ID']=''
 #       temp['COUNTRY']=''
 #       temp['DICT_RESULTS']=''
 #       temp['DOB']=''
 #       temp['GROUP']=''
 #       temp['ATHLETE_ID']=''
 #       temp['REACT_TIME']=''
 #       temp['FREE_FIELD']=''
 #       temp['FREE_FIELD2']=''
 #       temp['FREE_FIELD3']=''

        
  #      temp['WIND']=temp['test'].map(wind_dict) # map temp field across to wind data
        
  #      temp = temp[temp[0].str.contains("~~~ End of Listing ~~~") == False]  # drop useless rows
  #      temp = temp[temp[0].str.contains("W/Gauge :") == False]
  #      temp = temp[temp[0].str.contains("Pos") == False]
     #   temp = temp[temp[7].str.contains("DNS|NM|DQ|DNF") == False] # drop rows without results
        
            
        df_table=pd.concat([df_table, temp], axis=0)
         
        


#df_table.rename(
#    columns={0: "RANK", 1: "Tag", 2: "NAME", 3: "TEAM", 4: "RESULT", 5: "HEAT", 6: "LANE", 7: "REMARKS"},
#    inplace=True,
#)
        
#df_table.drop('test', axis=1, inplace=True)    # drop temp columns used to map wind data   

#df_table = df_table.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
#                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
#                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'])


#df_table.to_csv('sectrack_result_22a.csv', sep=',', index=False, encoding='utf-8')
        

            
                       
                       

        
            
                


reset 2
reset 2
reset 2
reset 2
Decathlon - 100m
reset 2
Discus Throw
reset 2
100m
reset 2
Decathlon Long Jump
reset 2
100m
reset 2
100m
reset 2
High Jump
reset 2
800m
reset 2
400m
reset 2
Decathlon Shot Put
reset 2
400m
reset 2
Decathlon High Jump
reset 2
400m Hurdles
reset 2
400m Hurdles
reset 2
Javelin Throw
reset 2
100m
reset 2
Triple Jump
reset 2
200m
reset 2
200m
reset 2
3000m
reset 2
1500m
reset 2
Decathlon 400m
reset 2
reset 2
reset 2
reset 2
10000m Race Walk
reset 2
Decathlon 110m Hurdles
reset 2
Long Jump
reset 2
reset 2
110m Hurdles
reset 2
Decathlon Discus Throw
reset 2
800m
reset 2
Shot Put
reset 2
200m
reset 2
reset 2
Discus Throw
reset 2
Decathlon Pole Vault
reset 2
reset 2
100m
reset 2
Decathlon Javelin Throw
reset 2
3000m Steeplechase
reset 2
Pole Vault
reset 2
reset 2
3000m Steeplechase
reset 2
400m
reset 2
400m
reset 2
5000m
reset 2
Decathlon 1500m
reset 2
100m
reset 2
100m
reset 2
DECATHLON
reset 2
reset 2
10000m Race Walk
reset 2
100m Hurdles
reset 2
Triple Jump
re

In [798]:
# Scrape U20 2024 PDFs
# For extracting pole vault results only -- Version 2

# TODO : correct data for first pole vault event, correct algo for picking up and correcting corrupted names

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/U20/All Results.pdf"

 
# Iterate over files in directory

event_pattern ='(MEN U20 -)|(WOMEN U20 -)'

heat_pattern = 'HEAT \d'

school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
year_pattern='\d\d\d\d'

#name_pattern='\d\d\d.{10-29}\d\d\d\d'

stage_pattern='FINAL|QUALIFYING|ROUND|SEMI|GROUP'

country_code = '\s[A-Z]{3}\s'

DOB_pattern='\s20[0-9]{2}\s'

#tag_pattern='\s[0-9]{3}\s'

tag_pattern='[0-9]{3}\s'

last_name_pattern = '[A-Z]{2,}'

wind_pattern='(\+\d\.\d|\-\d\.\d)'

name_pattern='[a-zA-Z\-\. \"]{3,31}'

results_begin = '^\d'  # line begins with a digit

react_time = '0\.[0-9]{3}'

rank_pattern = '^\d |^\dd '

qualification_pattern = ' Q | q '

relay_pattern = '\d\d\:\d\d\.\d\d'

corrupted_names = '[A-Z0-9]{5,}'

timing_pattern = '\d\d\.\d\d|\d.\d\d'

points_pattern = ' \d\d\d '





df=None

# Initialize empty df

df_results=pd.DataFrame(columns = ['RANK', 'TAG_ID', 'NAME', 'DOB', 'COUNTRY', 'RESULT', 'REMARKS', 'POINTS', 'DICT_RESULT'])

df_attempts=pd.DataFrame(columns = ['NAME', 'DICT_RESULT'])



with pdfplumber.open(file) as pdf:
        
        
    for i in range(len(pdf.pages)):    
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()        
        
        df=pd.DataFrame(text.splitlines())
        
                        
        for j in range(len(df)): # find row containing event details and column names  
            
            # Look for corrupted names and correct
            
         #   corrupted = re.findall(corrupted_names, df[0][j])
            
        
         #   print('corrupted', j, corrupted, df[0][j])
            
            df.loc[j, 0] = df.loc[j, 0].replace('M2a0d0u5shani HERSARTIH', 'Madushani HERSARTIH 2005 SRI')
            df.loc[j, 0] = df.loc[j, 0].replace('Kama2l 0A0Z8IRA', 'Kamal AZIRA 2008')
            df.loc[j, 0] = df.loc[j, 0].replace('AL SH2E0K0I5LI', 'AL SHEKILI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('ALDHUH20O0O7RI', 'ALDHUHOORI 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('KUWA20R0I7', 'KUWARI 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('KARLSSON2007', 'KARLSSON 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('EIDIYANKAKHAK2I005', 'EIDIYANKAKHAKI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('AGHAEI2005', 'AGHEI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('SWAKAR2007', 'SWAKAR 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('AKALANKA2006', 'AKALANKA 2006')
            df.loc[j, 0] = df.loc[j, 0].replace('ZADJA2LI005', 'ZADJALI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('ROSL2A0N07', 'ROSLAN 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('A ABDELSALAM2005', 'A ABDELSALAM 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('AL-H20A0JR6I', 'AL-HAJRI 2006')
            df.loc[j, 0] = df.loc[j, 0].replace('Abhish2e0k0a8', 'Abhisheka 2008')
            df.loc[j, 0] = df.loc[j, 0].replace('Al2 M00E8QBAALI', 'Al MEQBAALI 2008')
            df.loc[j, 0] = df.loc[j, 0].replace('ALSHANQ2IT0I07', 'ALSHANQITI 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('PRAMUDIK5A71', 'PRAMUDIKA')
            df.loc[j, 0] = df.loc[j, 0].replace('Jithm5a6 S9henarDi WunIuJEsTuHngUhNeG', 'Jithma Shenari Wunusunghe')
            df.loc[j, 0] = df.loc[j, 0].replace('THA3M40EL', 'THAMEL')
            df.loc[j, 0] = df.loc[j, 0].replace('T5h7e1mindaK RoAdiJtAhPuAwKaSkHkAu', 'Theminda Rodithuwakku')
            df.loc[j, 0] = df.loc[j, 0].replace('ALKATH20E0E6RI', 'ALKATHEERI 2006')
            df.loc[j, 0] = df.loc[j, 0].replace('Ahme2d0 0A7LMESSABIUAE', 'Ahmed ALMESSABIUAE 2007')

        
        for j in range(len(df)):
               
            
            if 'CHAMPIONSHIPS' in df[0][j]:
                event=df[0][j]
                
            print('event', event)
                                                
            if 'MEN' in df[0][j]:
                gender='Male'
                
            if 'WOMEN' in df[0][j]:
                gender='Female'   
                
             
           # Extract event info 
            
            if re.search(event_pattern, df[0][j]): # isolate event in two steps
                                
                epos = re.search(event_pattern, df[0][j])  # find row
                
                start = epos.end()
                
                string = df[0][j][start:] # strip out event part
                
                                
                try:
                    spos = re.search(stage_pattern, string)  # check if row contains stage indication
                    split_end = spos.start()-1
                    
                    event = string[:split_end]   # strip out stage
                    
                    stage = string[split_end:]
                    
                    event = event.strip()
                    
                    print(event)
                                        
                except:
                    
                    event = string.strip()    
                    
                    print(event)
                    
             # Extract Heat
            
            if 'HEAT' in df[0][j]:
                                
                heat=re.findall(heat_pattern, df[0][j])
                                
            # Extract group (for long jumps)
            
            if 'GROUP' in df[0][j]:
                
                group=df[0][j]
                
            
           
                

            if event=='Pole Vault' or event == 'Decathlon Pole Vault':   
                
                
                # Extract columns of table    

                
                if 'Pos.' in df[0][j] and 'BIB' in df[0][j]: # main result table
                            
                    columns=df[0][j]
                    column_names=columns.split(' ')
                                
                    column_names.remove('First')
                    column_names.remove('LAST')
                    column_names.remove('Name,')
                    
                if 'Pos.' in df[0][j] and 'BIB' not in df[0][j]: # attempts table
                            
                    columns=df[0][j]
                    column_names=columns.split(' ')
                                
                    column_names.remove('First')
                    column_names.remove('LAST')
                    column_names.remove('Name,')


            # Extract name
            
                if re.search(DOB_pattern, df[0][j]) and re.search(results_begin, df[0][j]):  # if line begins with a digit and dob pattern exists then a results line
                                                                  
                
                    find = re.findall(name_pattern, df[0][j])   # does not catch some names corrupted at the end
                                              
                    full_name = find[0]  
                    
            
                
            
            # Extract TAG ID
            
                res = re.findall(tag_pattern, df[0][j])
                            
                if len(res)>1:
                            
                    tag_id = res[0]
                
                
                else:
                
                    tag_id = res
                                
            
                        
            # Extract DOB
            
                dob = re.findall(DOB_pattern, df[0][j])
                                                                                      
                                           

            # Extract rank, country, result and remarks. 
            # Look for existance of tag_id, dob and rank to find valid line
            
            
                if re.search(DOB_pattern, df[0][j]) and re.search(tag_pattern, df[0][j]) and re.search(rank_pattern, df[0][j]):  # lines with DQ and DNS are not picking up because tag_id for those has no space in front
                
                    print(df[0][j])
                
                    result_list = df[0][j].split(' ')
                    
                    rank=result_list[0]
                                        
                    
                                        
                    if result_list[-1] =='SB' or result_list[-1]=='PB' and event=='Pole Vault':  # extract PB/SB
                        
                            remarks = result_list[-1]
                            
                            
                            result = result_list[-2]  # result is second last element
                            
                            
                            country = result_list[-3] # country is third last element
                        
                            result_list.pop()  
                        
                    if result_list[-1] !='SB' or result_list[-1]!='PB' and event=='Decathlon Pole Vault': 
                          
                                    
                        remarks = ' '
                            
                        result = result_list[-2]  # result is second last element
                           
                        country = result_list[-3] # country is 3rd last element
                        
                        
                        points = result_list[-1] # points is the last element 
                        
                                                                        
                
                    
                    print('df entry', event, rank, tag_id, full_name, dob, country, reaction, result, group, dict_result, points, qualification, heat, stage, gender, remarks)

                    temp_list.append(rank)
                    temp_list.append(tag_id)
                    temp_list.append(full_name)
                    temp_list.append(dob)
                    temp_list.append(country)
                    temp_list.append(result)
                    temp_list.append(remarks)
                    temp_list.append(points)
                    temp_list.append(' ')
                    
                    print('temp list', temp_list)
                
                                
                # append to dataframe    
                    
                    df_results.loc[len(df_results)] = temp_list

           ######################################################### 
            
            
                    
                # Set keys for dictionary
                        
                idx = column_names.index('NAME')
                
                keys = column_names[idx+1:]  #  attempts come one after 'results' column
                                
                len_keys = len(keys)
                        
                # Assign values to keys
                                        
                attempts = df[0][j].split(' ')
                
                        
                        
                    
            
            
            ################################################################## 
              # When the attempts table is reached
                
                if column_names[-1]=='Result':   # plain pole vault event
                    
                    print('here')
    
    
            #######################################################################
                set_list = ['-', 'X', 'O', 'XO', 'XXO', 'XXX']

                
                if re.search(name_pattern, df[0][j]) and re.search(results_begin, df[0][j]):  # reset full name if attempts table is reached
                                                                  
                    attempts_line = df[0][j]
                    
                    print('attempts line', attempts_line)
                    
                    
                    name_find = re.findall(name_pattern, df[0][j])   # does not catch some names corrupted at the end
                                              
                    name_find = name_find[0] 
                    
                    
                    attempt_name = name_find.replace("-","")
                    
                    attempt_name = attempt_name.replace("O", "")
                    
                    attempt_name = attempt_name.replace("X", "")
                    
                    attempt_name = attempt_name.strip()
                    
                    print('attempt name', attempt_name)
                    
                          
                
                list_dict = []
                                                
                       
                idx = column_names.index('NAME')
                
                keys = column_names[idx+1:]  #  attempts come one after 'results' column
                
                print('keys', keys)
                
                len_keys = len(keys)
                                                
                  
        #        if 'Pos.' in column_names and 'Born' not in column_names and 'BIB' not in column_names:
                    
        #            print('found it')
        
               
                if re.search(results_begin, df[0][j]) and re.search(name_pattern, df[0][j]) and not re.search(tag_pattern, df[0][j]) and not re.search(DOB_pattern, df[0][j]):  # identify list of attempts as no dob and no tag_id
                
                    values = df[0][j].split(' ')
                                                            
                    
            #        print('values', values)
                    
                    attempts_values = values[-len_keys:] # extract backwards using length of keys
                    
                    print('attempts values', attempts_values)
                                         
                    
                    # if attempts and keys are same list (meaning encountered a header line)then skip
                        
                    if attempts_values == keys:
                        
                        attempts = []
                        
                    else:
                        
                        attempts = attempts_values
                                                
                        # make sure attempts contains only '-', 'O', 'X' 'XXX', etc.
                                                
                        res = [x for x in attempts if x in set_list]
                        
                        attempts_only = res
                        
                        print('attempts only', attempts_only)
                        
                                                
                        # need to match name and append
                        
                        
                    # create dictionary of attempts
                    
                    dict_pole = dict(zip(keys, attempts_only))
                                        
                    dict_result = dict_pole
                    
                    print('dict result', dict_result)
                    
                    list_dict.append(attempt_name)
                    list_dict.append(dict_result)
                    
                #    list_dict = dict(zip(attempt_name, dict_result))
                    
                                                    
             #       list_of_list.append(list_dict)
                    
             #   temp_list.append(attempt_name)    
             #   temp_list.append(list_of_list)    
                
                print('list dict', list_dict)
                
                if list_dict!=[]:
                    
                    df_attempts.loc[len(df_attempts)] = list_dict
                           
        #        print('list of list', list_of_list)
               
        
            # reset variables for next loop
            
                
            points = None
        #    reaction = None
            wind = None
            country = None
            dob = None
            full_name = None
            tag_id = None
            rank = None
         #   result = None
         #   heat = None
            res_dict = None
            remarks = None
            group = None
            qualification = None
            name_list = []
            
            temp_list=[]
        
            list_of_list = []
            
            
     #   relay_names = []
        
        
        event = None
        result = None
        reaction = None
        heat = None
        dict_result = None

        
        

event None
event None
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS 2024
event 21ST ASIAN U20 ATHLETI

event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event Triple Jump
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 2

event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 100m
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
Decathlon Javelin Throw
event Decathlon Javelin Throw
event Decathlon Javelin Throw
event Decathlon Javelin Throw
event Decathlon Javelin Throw
event Decathlon Javelin Throw
event Decathlon Javelin Throw
event Decathlon Javelin Throw
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAM

event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event DECATHLON
event None
event None
event None
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
10000m Race Walk
event 10000m Race Walk
event 10000m Race Walk
event 10000m Race Walk
event 10000m Race Walk
event 10000m Race Walk
event 10000m Race Walk
event 10000m Race Walk
event 10000m Race Walk
event 10000m Race Walk
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
100m Hurdles
event 100m Hurdles
event 100m Hur

event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event Shot Put
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
Heptathlon 200m
event Heptathlon 200m
event Heptathlon 200m
event Heptathlon 200m
event Heptathlon 200m
event Heptathlon 200m
event Heptathlon 200m
event Heptathlon 200m
event Heptathlon 200m
event Heptathlon 200m
event Heptathlon 200m
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200m
event 200

event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
event 21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON
event HEPTATHLON


In [799]:
df_results

,RANK,TAG_ID,NAME,DOB,COUNTRY,RESULT,REMARKS,POINTS,DICT_RESULT
0,1,384,Nodir NORBOEV,[ 2005 ],UZB,4.10,,645,
1,2,220,Hyunbin NAM,[ 2005 ],KOR,3.70,,535,
2,3,208,Sening IVAN,[ 2006 ],KAZ,3.30,,431,
3,4,200,Kotov MIKHAIL,[ 2006 ],KAZ,3.20,,406,
4,1,503,Yanagawa MIKU,[ 2006 ],2006,JPN,,3.85,
5,2,507,Cherkashina ANNA,[ 2008 ],2008,KAZ,,3.60,
6,3,442,Maria Andriani MELABESSY,[ 2006 ],2006,INA,,3.60,
7,4,511,Kabirova ELMIRA,[ 2006 ],2006,KAZ,,3.45,
8,5,562,Esther Tay Shee WEI,[ 2005 ],2005,SGP,,3.30,
9,1,314,Seifeldin Mohamed A ABDELSALAM,[ 2005 ],2005,QAT,,5.51,


In [801]:
df_attempts

,NAME,DICT_RESULT
0,Nodir NRBEV,"{'3.00': '-', '3.10': '-', '3.20': '-', '3.30'..."
1,Hyunbin NAM,"{'3.00': '-', '3.10': '-', '3.20': '-', '3.30'..."
2,Sening IVAN,"{'3.00': 'O', '3.10': '-', '3.20': 'X', '3.30'..."
3,Kotov MIKHAIL,"{'3.00': 'O', '3.10': '-', '3.20': 'X', '3.30'..."
4,Yanagawa MIKU,"{'2.80': 'O', '3.00': 'O', '3.15': 'X', '3.30'..."
5,Cherkashina ANNA,"{'2.80': '-', '3.00': '-', '3.15': '-', '3.30'..."
6,Maria Andriani MELABESSY,"{'2.80': '-', '3.00': '-', '3.15': 'X', '3.30'..."
7,Kabirova ELMIRA,"{'2.80': 'X', '3.00': 'X', '3.15': 'O', '3.30'..."
8,Esther Tay Shee WEI,"{'2.80': '-', '3.00': '-', '3.15': 'X', '3.30'..."
9,Seifeldin Mohamed A ABDELSALAM,"{'3.80': '-', '3.95': '-', '4.00': '-', '4.05'..."


In [888]:
# Scrape U20 2024 PDFs
# For extracting shot put, discus throw and javelin throw format results only

# TODO : correct data for first pole vault event, correct algo for picking up and correcting corrupted names

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/U20/All Results.pdf"

 
# Iterate over files in directory

event_pattern ='(MEN U20 -)|(WOMEN U20 -)'

heat_pattern = 'HEAT \d'

school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
year_pattern='\d\d\d\d'

#name_pattern='\d\d\d.{10-29}\d\d\d\d'

stage_pattern='FINAL|QUALIFYING|ROUND|SEMI|GROUP'

country_code = '\s[A-Z]{3}\s'

DOB_pattern='\s20[0-9]{2}\s'

#tag_pattern='\s[0-9]{3}\s'

tag_pattern='[0-9]{3}\s'

last_name_pattern = '[A-Z]{2,}'

wind_pattern='(\+\d\.\d|\-\d\.\d)'

name_pattern='[a-zA-Z\-\. \"]{3,31}'

results_begin = '^\d'  # line begins with a digit

react_time = '0\.[0-9]{3}'

rank_pattern = '^\d |^\dd '

qualification_pattern = ' Q | q '

relay_pattern = '\d\d\:\d\d\.\d\d'

corrupted_names = '[A-Z0-9]{5,}'

timing_pattern = '\d\d\.\d\d|\d.\d\d'




df_table=pd.DataFrame()  # initialize empty master df

df=None

df_results=pd.DataFrame(columns = ['EVENT', 'COMPETITION', 'RANK', 'TAG_ID', 'NAME', 'DOB', 'COUNTRY', 'RESULT', 'REMARKS', 'GENDER', 'POINTS', 'DICT_RESULT'])

df_temp=pd.DataFrame(columns = ['NAME', 'DICT_RESULT'])


with pdfplumber.open(file) as pdf:
        
        
    for i in range(len(pdf.pages)):    
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()        
        
        df=pd.DataFrame(text.splitlines())
        
                        
        for j in range(len(df)): # find row containing event details and column names  
            
            # Look for corrupted names and correct
            
            corrupted = re.findall(corrupted_names, df[0][j])
            
                
            df.loc[j, 0] = df.loc[j, 0].replace('M2a0d0u5shani HERSARTIH', 'Madushani HERSARTIH 2005 SRI')
            df.loc[j, 0] = df.loc[j, 0].replace('Kama2l 0A0Z8IRA', 'Kamal AZIRA 2008')
            df.loc[j, 0] = df.loc[j, 0].replace('AL SH2E0K0I5LI', 'AL SHEKILI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('ALDHUH20O0O7RI', 'ALDHUHOORI 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('KUWA20R0I7', 'KUWARI 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('KARLSSON2007', 'KARLSSON 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('EIDIYANKAKHAK2I005', 'EIDIYANKAKHAKI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('AGHAEI2005', 'AGHEI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('SWAKAR2007', 'SWAKAR 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('AKALANKA2006', 'AKALANKA 2006')
            df.loc[j, 0] = df.loc[j, 0].replace('ZADJA2LI005', 'ZADJALI 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('ROSL2A0N07', 'ROSLAN 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('A ABDELSALAM2005', 'A ABDELSALAM 2005')
            df.loc[j, 0] = df.loc[j, 0].replace('AL-H20A0JR6I', 'AL-HAJRI 2006')
            df.loc[j, 0] = df.loc[j, 0].replace('Abhish2e0k0a8', 'Abhisheka 2008')
            df.loc[j, 0] = df.loc[j, 0].replace('Al2 M00E8QBAALI', 'Al MEQBAALI 2008')
            df.loc[j, 0] = df.loc[j, 0].replace('ALSHANQ2IT0I07', 'ALSHANQITI 2007')
            df.loc[j, 0] = df.loc[j, 0].replace('PRAMUDIK5A71', 'PRAMUDIKA')
            df.loc[j, 0] = df.loc[j, 0].replace('Jithm5a6 S9henarDi WunIuJEsTuHngUhNeG', 'Jithma Shenari Wunusunghe')
            df.loc[j, 0] = df.loc[j, 0].replace('THA3M40EL', 'THAMEL')
            df.loc[j, 0] = df.loc[j, 0].replace('T5h7e1mindaK RoAdiJtAhPuAwKaSkHkAu', 'Theminda Rodithuwakku')
            df.loc[j, 0] = df.loc[j, 0].replace('ALKATH20E0E6RI', 'ALKATHEERI 2006')
            df.loc[j, 0] = df.loc[j, 0].replace('Ahme2d0 0A7LMESSABIUAE', 'Ahmed ALMESSABIUAE 2007')

            
        # Extract competition and gender   
            
            if 'CHAMPIONSHIPS' in df[0][j]:
                competition=df[0][j]
                
            print('event', event)
                                                
            if 'MEN' in df[0][j]:
                gender='Male'
                
            if 'WOMEN' in df[0][j]:
                gender='Female'   
             
        # Extract event
                    
            if re.search(event_pattern, df[0][j]): # isolate event in two steps
                                
                epos = re.search(event_pattern, df[0][j])  # find row
                
                start = epos.end()
                
                string = df[0][j][start:] # strip out event part
                
                                
                try:
                    spos = re.search(stage_pattern, string)  # check if row contains stage indication
                    split_end = spos.start()-1
                    
                    event = string[:split_end]   # strip out stage
                    
                    stage = string[split_end:]
                    
                    event = event.strip()
                    
                    print(event)
                                        
                except:
                    
                    event = string.strip()    
                    
                    print(event)
                    
             # Extract Heat
            
            if 'HEAT' in df[0][j]:
                                
                heat=re.findall(heat_pattern, df[0][j])
                                
            # Extract group (for long jumps)
            
            if 'GROUP' in df[0][j]:
                
                group=df[0][j]
                
            # Extract columns of table    
            
            if 'Pos' in df[0][j]: # extract column names
                            
                row_index=index  # where the header row is
                columns=df[0][j]
                column_names=columns.split(' ')
                                
                column_names.remove('First')
                column_names.remove('LAST')
                column_names.remove('Name,')            

           
                

            if event=='Shot Put' or event=='Heptathlon Shot Put' or event == 'Javelin Throw' or event == 'Discus Throw' or event=='Hammer Throw':   
                
                              
            # Extract alpha 3 country code. Look for it after DOB
            
            
                try:
                                            
                    find = re.search(DOB_pattern, df[0][j])
                                
                    start = find.end()
                    end = find.end()+3
            
                    country = df[0][j][start:end].strip()
                                                            
                    if country in list_countries:
                       
                        pass
                
                    else:
                    
                        country = None
                
                
                except:
                
                    pass
                
                
            
            # Extract TAG ID
            
                res = re.findall(tag_pattern, df[0][j])
                            
                if len(res)>1:
                            
                    tag_id = res[0]
                
                
                else:
                
                    tag_id = res
                                
            
                        
            # Extract DOB
            
                dob = re.findall(DOB_pattern, df[0][j])
                                        
                                    
            
            # Extract name (will pick up nation code as a 3rd element in list)
                        
          #  if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                
            
                if re.search(DOB_pattern, df[0][j]) and re.search(results_begin, df[0][j]):  # if line begins with a digit and dob pattern exists then a results line
                                                                  
                    find = re.findall(name_pattern, df[0][j])   # does not catch some names corrupted at the end
                    
                                              
                    full_name = find[0] 
                                                                                                
                                               

            # Extract result. Put into a list, search for country code index and choose index after
            # Look for existance of tag_id, dob to find valid line
            
            
                
                
                    print('index', index)
                                                
                    print('row', df[0][j])
                                
                    idx = column_names.index('Result')
                
                    keys = column_names[idx+1:]  #  attempts come one after 'results' columns   
                
                    print('keys', keys)
                              
                
                    if re.search(DOB_pattern, df[0][j]) and re.search(tag_id, df[0][j]):  # if a results line
                        
                        
                        result_list=df[0][j].split(' ')
                    
                        rank =  result_list[0] # extract rank      
                        
                        name_match = re.findall(name_pattern, df[0][j])   # extract name
                                              
                        full_name = name_match[0]  # extract name
                                        
                                                        
                        country_idx = result_list.index(country)
                        
                        result = result_list[country_idx+1]  # extract result
                                        
                        attempt_values = result_list[country_idx+2:]  # extract list of attempts
                    
                                        
        
    #                if re.search('DOB_pattern', str(values[0])):  # first values is DOB choose next values onwards
                                 
    #                    values = result_list[result_index+3:]
                                
    #                    print('new values', values)        
            
                                        
                                        
                        if attempt_values[-1] =='SB' or attempt_values[-1]=='PB':  # extract PB/SB
                        
                            remarks = attempt_values[-1]
                        
                            attempt_values.pop()  
                        
                            print('remarks', remarks)
                        
                        try:
                        
                            if attempt_values[0] == 'Q' or attempt_values[0] == 'q': # extract Q/q
                        
                                qualification = attempt_values[0]
                        
                                attempt_values.pop(0)
                            
                        except:
                        
                            pass
                    
                        print('attempt values', attempt_values)
                    
                    
                    
            
             
               
                 # zip wind and results lists into two values per key dictionary
                    print('name', full_name)

                    print('keys', keys)
            
                    print('result', result)
    
                    dict_result = dict(zip(keys, attempt_values)) 
        
                    print('dict', dict_result)
                    
                    temp_list.append(event)
                    temp_list.append(competition)
                    temp_list.append(rank)
                    temp_list.append(tag_id)
                    temp_list.append(full_name)
                    temp_list.append(dob)
                    temp_list.append(country)
                    temp_list.append(result)
                    temp_list.append(remarks)
                    temp_list.append(gender)
                    temp_list.append(points)
                    temp_list.append(dict_result)
                    
                    print('temp list', temp_list)
                                    
                                
                # append to dataframe    
                    
                    df_results.loc[len(df_results)] = temp_list

        
                    
                    print('df entry', event, competition, rank, tag_id, full_name, dob, country, wind, reaction, result, group, dict_result, points, qualification, heat, stage, gender, remarks)
                    
               
            # append result into dataframe
        
        
  #          if event == '400m':
        
   #             print(event, rank, tag_id, full_name, dob, country, wind, reaction, result, group, res_dict, points, qualification, heat, stage, gender, remarks)

    
                    
        
            # reset variables for next loop
            
            print('reset 1')
                
            points = None
        #    reaction = None
            wind = None
            country = None
            dob = None
            full_name = None
            tag_id = None
            rank = None
         #   result = None
         #   heat = None
            res_dict = None
            remarks = None
            group = None
            qualification = None
            name_list = []
        
            list_of_list = []
            
            temp_list=[]

        print('reset 2')
            
     #   relay_names = []
        
        
        event = None
        result = None
        reaction = None
        heat = None
        dict_result = None

        
        

event None
reset 1
event None
reset 1
reset 2
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
reset 2
event None
reset 1
event None
reset 1
reset 2
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event N

event None
reset 1
event None
reset 1
event None
100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
reset 2
event None
reset 1
event None
reset 1
event None
reset 1
event None
High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
event High Jump
reset 1
reset 2
event None
reset 1
event None
reset 1
event None
reset 1
event None
800m
reset 1
event 800m
reset 1
event 800m
reset 1
event 800m
reset 1
event 800m
reset 1
event 800m
reset 1
event 800m
reset 1
event 800m
reset 1
event 800m
reset 1
event 800m
reset 1
event

event None
reset 1
event None
reset 1
event None
reset 1
event None
Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
event Triple Jump
reset 1
reset 2
event None
reset 1
event None
reset 1
event None
reset 1
event None
200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
even

event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
event 200m
reset 1
reset 2
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
event None
reset 1
reset 2
event None
reset 1
event None
reset 1
event None
reset 1
event None
Discus Throw
reset 1
event Discus Throw
re

event None
reset 1
event None
reset 1
event None
reset 1
event None
Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
event Decathlon Pole Vault
reset 1
reset 2
event None
reset 1
event None
reset 1
event None
reset 1
reset 2
event None
reset 1
event None
reset 1
event None
reset 1
event None
100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
event 100m
reset 1
eve

event None
reset 1
event None
reset 1
event None
4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m Relay
reset 1
event 4x100m

IndexError: list index out of range

In [889]:
df_results.tail(50)

,EVENT,COMPETITION,RANK,TAG_ID,NAME,DOB,COUNTRY,RESULT,REMARKS,GENDER,POINTS,DICT_RESULT
16,Shot Put,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,6,212,Uteshev KONSTANTIN,[ 2007 ],KAZ,14.83,None,Male,None,"{'1': '14.04', '2': '14.06', '3': '14.83', '4'..."
17,Shot Put,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,7,129,Afrizal Faiz Aska PUTRA,[ 2006 ],INA,14.03,None,Male,None,"{'1': '13.47', '2': '13.52', '3': '12.80', '4'..."
18,Shot Put,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,8,373,Salem Yousufsalem Mohammad Al,[ 2008 ],UAE,12.76,None,Male,None,"{'1': '12.73', '2': 'X', '3': '12.76', '4': 'X..."
19,Shot Put,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,9,298,Djibrine Adoum D Ahmat AHMAT,[ 2005 ],QAT,NM,None,Male,None,"{'1': 'X', '2': 'X', '3': 'X'}"
20,Discus Throw,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,1,404,Bingyang HAN,[ 2007 ],CHN,56.35,PB,Female,None,"{'1': '55.82', '2': 'X', '3': '55.43', '4': '5..."
21,Discus Throw,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,2,447,Amanat KAMBOJ,[ 2007 ],IND,50.45,PB,Female,None,"{'1': '50.45', '2': 'X', '3': '47.69', '4': 'X..."
22,Discus Throw,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,3,411,Mengyuan ZHANG,[ 2006 ],CHN,50.13,None,Female,None,"{'1': '46.40', '2': '50.13', '3': '48.78', '4'..."
23,Discus Throw,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,4,458,Nikita KUMARI,[ 2006 ],IND,46.84,PB,Female,None,"{'1': 'X', '2': 'X', '3': '44.25', '4': 'X', '..."
24,Discus Throw,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,5,531,Hyemin LEE,[ 2006 ],KOR,46.62,None,Female,None,"{'1': '46.62', '2': 'X', '3': 'X', '4': 'X', '..."
25,Discus Throw,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS,6,611,Yulianna SHCHUKINA,[ 2005 ],UZB,44.20,PB,Female,None,"{'1': '39.77', '2': '42.02', '3': '42.32', '4'..."
